In [1]:
import re

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [2]:
# Read in both CSV files
file1_path = "data/Parcel_Data_2021_Table_-2691831558175163259.csv"
file2_path = "data/TTC_Secured_Property_Taxes.csv"

# Read the CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Display basic info about the dataframes
print(f"Dataset 1 shape: {df1.shape}")
print(f"Dataset 2 shape: {df2.shape}")


/var/folders/nc/46rcs7vd2z304swddlmzv9280000gn/T/ipykernel_35042/2178698503.py:12: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(file1_path)
/var/folders/nc/46rcs7vd2z304swddlmzv9280000gn/T/ipykernel_35042/2178698503.py:13: DtypeWarning: Columns (29,30,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(file2_path)


Dataset 1 shape: (2424770, 51)
Dataset 2 shape: (2571799, 69)


In [3]:
df1[df1["Property Location"].str.startswith("1017 HYPERION AVE", na=False)]


,Zip Code,City Tax Rate Area,AIN,Roll Year,Tax Rate Area Code,Assessor ID,Property Location,Property Use Type,Property Use Code,Use Code 1st Digit,...,Address House Number Fraction,Direction,Street,Unit Number,City,Zip Code.1,Row ID,Location Latitude,Location Longitude,OBJECTID
1252963,90029-3109,LOS ANGELES,5427007013,2024,13,5427-007-013,1017 HYPERION AVE LOS ANGELES CA 90029,R-I,0200,Residential,...,,,HYPERION AVE,,LOS ANGELES CA,90029.0,20245427007013,34.090412,-118.280831,56340440


In [4]:
df1["City Tax Rate Area"].value_counts()

City Tax Rate Area
LOS ANGELES       805738
unincorporated    326824
LONG BEACH        106381
SANTA CLARITA      71910
LANCASTER          50077
                   ...  
ROSEMEAD             269
MAYWOOD              213
PARAMOUNT             94
INDUSTRY              36
EL SEGUNDO             4
Name: count, Length: 123, dtype: int64

In [5]:
df2 = df2.loc[df2.groupby("PARCEL_NUMBER")["PARCEL_HISTORY_YRSEQ"].idxmax()]


In [6]:
df2[df2["PARCEL_NUMBER"] == 5427007013]


,RECORD_TYPE,PARCEL_YEAR_8900S,PARCEL_NUMBER,YEAR_ACTIVE_TABLE,MINING_RIGHTS_KEY,TAX_STATUS_KEY,YEAR_TAX_DEFAULTED,SENIOR_CITIZEN_YEAR,F4PAY,F500_ACCOUNT_NUMBER,...,HOMEOWNERS_EXEMPTION,MOBILE_OWNER_EXEMPTION,FIXTURES_EXEMPTION,PERSONAL_PROPERTY_EXEMPTION,TAX_TYPE_1_AMOUNT,TAXTYPE_2_AMOUNT,TAX_TYPE_3_AMOUNT,TAX_TYPE_4_AMOUNT,TAX_TYPE_8_AMOUNT,OBJECTID
1312298,1,NaN,5427007013,111111111,NaN,0,0,0,0,0,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0,0.0,1312299


In [7]:
# Merge the dataframes - df1 is the left dataframe, and we're matching AIN from df1
# with PARCEL_NUMBER from df2
merged_df = pd.merge(
    left=df1, right=df2, left_on="AIN", right_on="PARCEL_NUMBER", how="left"
)

# Display info about the merged dataframe
print(f"Merged dataset shape: {merged_df.shape}")

# Preview the merged dataframe
print(merged_df.head())

Merged dataset shape: (2424770, 120)
     Zip Code City Tax Rate Area         AIN  Roll Year  Tax Rate Area Code  \
0  91304-3327        LOS ANGELES  2004001003       2024                  16   
1  91304-3327        LOS ANGELES  2004001004       2024                  16   
2  91304-3327        LOS ANGELES  2004001005       2024                  16   
3  91304-3332        LOS ANGELES  2004001008       2024                  16   
4  91304-3332        LOS ANGELES  2004001009       2024                  16   

    Assessor ID                        Property Location Property Use Type  \
0  2004-001-003    8321 FAUST AVE  LOS ANGELES CA  91304               SFR   
1  2004-001-004    8313 FAUST AVE  LOS ANGELES CA  91304               SFR   
2  2004-001-005    8309 FAUST AVE  LOS ANGELES CA  91304               SFR   
3  2004-001-008  8325 MAYNARD AVE  LOS ANGELES CA  91304               SFR   
4  2004-001-009  8311 MAYNARD AVE  LOS ANGELES CA  91304               SFR   

  Property Use Code

In [8]:
merged_df = merged_df[merged_df["City Tax Rate Area"] == "LOS ANGELES"]
merged_df

,Zip Code,City Tax Rate Area,AIN,Roll Year,Tax Rate Area Code,Assessor ID,Property Location,Property Use Type,Property Use Code,Use Code 1st Digit,...,HOMEOWNERS_EXEMPTION,MOBILE_OWNER_EXEMPTION,FIXTURES_EXEMPTION,PERSONAL_PROPERTY_EXEMPTION,TAX_TYPE_1_AMOUNT,TAXTYPE_2_AMOUNT,TAX_TYPE_3_AMOUNT,TAX_TYPE_4_AMOUNT,TAX_TYPE_8_AMOUNT,OBJECTID_y
0,91304-3327,LOS ANGELES,2004001003,2024,16,2004-001-003,8321 FAUST AVE LOS ANGELES CA 91304,SFR,0101,Residential,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,1.0
1,91304-3327,LOS ANGELES,2004001004,2024,16,2004-001-004,8313 FAUST AVE LOS ANGELES CA 91304,SFR,0101,Residential,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,2.0
2,91304-3327,LOS ANGELES,2004001005,2024,16,2004-001-005,8309 FAUST AVE LOS ANGELES CA 91304,SFR,0100,Residential,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,3.0
3,91304-3332,LOS ANGELES,2004001008,2024,16,2004-001-008,8325 MAYNARD AVE LOS ANGELES CA 91304,SFR,0101,Residential,...,7000.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,4.0
4,91304-3332,LOS ANGELES,2004001009,2024,16,2004-001-009,8311 MAYNARD AVE LOS ANGELES CA 91304,SFR,0101,Residential,...,7000.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016313,90732-4530,LOS ANGELES,7563037052,2024,20,7563-037-052,2193 WARMOUTH ST LOS ANGELES CA 90732,SFR,0100,Residential,...,7000.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,2110629.0
2016314,90732-4519,LOS ANGELES,7563037053,2024,20,7563-037-053,3810 S ANCHOVY AVE LOS ANGELES CA 90732,SFR,0100,Residential,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,2110630.0
2016315,90732-4519,LOS ANGELES,7563037054,2024,20,7563-037-054,3806 S ANCHOVY AVE LOS ANGELES CA 90732,SFR,0100,Residential,...,7000.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,2110631.0
2016316,90732-4556,LOS ANGELES,7563037055,2024,20,7563-037-055,2197 W PASEO DEL MAR LOS ANGELES CA 90732,SFR,0100,Residential,...,7000.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,2110632.0


In [9]:
# Option 1: Set the display option to show all columns
pd.set_option("display.max_columns", None)

print(list(merged_df.columns))

['Zip Code', 'City Tax Rate Area', 'AIN', 'Roll Year', 'Tax Rate Area Code', 'Assessor ID', 'Property Location', 'Property Use Type', 'Property Use Code', 'Use Code 1st Digit', 'Use Code 2nd Digit', 'Use Code 3rd Digit', 'Use Code 4th Digit', 'Number of Buildings', 'Year Built', 'Effective Year', 'Square Footage', 'Number of Bedrooms', 'Number of Bathrooms', 'Number of Units', 'Recording Date', 'Land Value', 'Land Base Year', 'Improvement Value', 'Improvement Base Year', 'Total Value, Land + Improvement', 'Home Owners Exemption', 'Real Estate Exemption', 'Fixture Value', 'Fixture Exemption', 'Personal Property Value', 'Personal Property Exemption', 'Property taxable?', 'Total Value', 'Total Exemption', 'Taxable Value', 'Classification', 'Region Number', 'Cluster Code', 'Parcel Legal Description', 'Address House Number', 'Address House Number Fraction', 'Direction', 'Street', 'Unit Number', 'City', 'Zip Code.1', 'Row ID', 'Location Latitude', 'Location Longitude', 'OBJECTID_x', 'RECORD_

In [10]:
selected = merged_df[
    [
        "Zip Code",
        "City Tax Rate Area",
        "AIN",
        "Roll Year",
        "Tax Rate Area Code",
        "Property Location",
        "Property Use Type",
        "Property Use Code",
        "Number of Buildings",
        "Year Built",
        "Effective Year",
        "Square Footage",
        "Number of Bedrooms",
        "Number of Bathrooms",
        "Number of Units",
        "Land Value",
        "Land Base Year",
        "Improvement Value",
        "Improvement Base Year",
        "Total Value, Land + Improvement",
        "Home Owners Exemption",
        "Real Estate Exemption",
        "Fixture Value",
        "Fixture Exemption",
        "Personal Property Value",
        "Personal Property Exemption",
        "Property taxable?",
        "Total Value",
        "Total Exemption",
        "Taxable Value",
        "Classification",
        "Region Number",
        "Location Latitude",
        "Location Longitude",
        "F1ST_INSTALLMENT_TAX",
        "F1ST_INSTALLMENT_BALANCE_DUE",
        "F1ST_INSTALLMENT_PENALTY",
        "F1ST_INSTALLMENT_PENALTY_PAID",
        "F1ST_INSTALLMENT_PAID_DATE",
        "F2ND_INSTALLMENT_TAX",
        "F2ND_INSTALLMENT_BALANCE_DUE",
        "F2ND_INSTALLMENT_PENALTY",
        "F2ND_INSTALLMENT_PENALTY_PAID",
        "F2ND_INSTALLMENT_PAID_DATE",
        "COST_DUE",
        "COST_PAID",
        "NONSUFFICIENT_FUNDS_DUE",
        "NONSUFFICIENT_FUNDS_PAID",
        "TOTAL_TAX_DUE",
        "TAX_TYPE_1_AMOUNT",
        "TAXTYPE_2_AMOUNT",
        "TAX_TYPE_3_AMOUNT",
        "TAX_TYPE_4_AMOUNT",
        "TAX_TYPE_8_AMOUNT",
    ]
]

In [11]:
selected["Total_Taxes_Paid_Calc"] = (
    selected["F1ST_INSTALLMENT_TAX"] + selected["F2ND_INSTALLMENT_TAX"]
)
selected

/var/folders/nc/46rcs7vd2z304swddlmzv9280000gn/T/ipykernel_35042/528749273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected["Total_Taxes_Paid_Calc"] = (


,Zip Code,City Tax Rate Area,AIN,Roll Year,Tax Rate Area Code,Property Location,Property Use Type,Property Use Code,Number of Buildings,Year Built,Effective Year,Square Footage,Number of Bedrooms,Number of Bathrooms,Number of Units,Land Value,Land Base Year,Improvement Value,Improvement Base Year,"Total Value, Land + Improvement",Home Owners Exemption,Real Estate Exemption,Fixture Value,Fixture Exemption,Personal Property Value,Personal Property Exemption,Property taxable?,Total Value,Total Exemption,Taxable Value,Classification,Region Number,Location Latitude,Location Longitude,F1ST_INSTALLMENT_TAX,F1ST_INSTALLMENT_BALANCE_DUE,F1ST_INSTALLMENT_PENALTY,F1ST_INSTALLMENT_PENALTY_PAID,F1ST_INSTALLMENT_PAID_DATE,F2ND_INSTALLMENT_TAX,F2ND_INSTALLMENT_BALANCE_DUE,F2ND_INSTALLMENT_PENALTY,F2ND_INSTALLMENT_PENALTY_PAID,F2ND_INSTALLMENT_PAID_DATE,COST_DUE,COST_PAID,NONSUFFICIENT_FUNDS_DUE,NONSUFFICIENT_FUNDS_PAID,TOTAL_TAX_DUE,TAX_TYPE_1_AMOUNT,TAXTYPE_2_AMOUNT,TAX_TYPE_3_AMOUNT,TAX_TYPE_4_AMOUNT,TAX_TYPE_8_AMOUNT,Total_Taxes_Paid_Calc
0,91304-3327,LOS ANGELES,2004001003,2024,16,8321 FAUST AVE LOS ANGELES CA 91304,SFR,0101,1,1973,1973,2090,4,3,1,711689,2006,301176,2006,1012865,0,0,0,0,0,0,Y,1012865,0,1012865,,2,34.220225,-118.620681,6326.89,0.0,0.0,0.0,11/05/2024,6326.87,0.00,0.0,0.0,02/03/2025,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,12653.76
1,91304-3327,LOS ANGELES,2004001004,2024,16,8313 FAUST AVE LOS ANGELES CA 91304,SFR,0101,1,1973,1973,2479,5,3,1,370538,2010,255879,2010,626417,0,0,0,0,0,0,Y,626417,0,626417,,2,34.220044,-118.620681,4017.94,0.0,0.0,0.0,12/10/2024,4017.94,4017.94,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,8035.88
2,91304-3327,LOS ANGELES,2004001005,2024,16,8309 FAUST AVE LOS ANGELES CA 91304,SFR,0100,1,1973,1973,2057,4,2,1,526360,2018,198577,2018,724937,0,0,0,0,0,0,Y,724937,0,724937,,2,34.219862,-118.620688,4569.88,0.0,0.0,0.0,11/20/2024,4569.88,4569.88,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,9139.76
3,91304-3332,LOS ANGELES,2004001008,2024,16,8325 MAYNARD AVE LOS ANGELES CA 91304,SFR,0101,1,1978,1978,2423,4,3,1,128421,1980,221965,1980,350386,7000,0,0,0,0,0,Y,350386,7000,343386,,2,34.220339,-118.622718,2354.27,0.0,0.0,0.0,12/11/2024,2354.26,2354.26,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,4708.53
4,91304-3332,LOS ANGELES,2004001009,2024,16,8311 MAYNARD AVE LOS ANGELES CA 91304,SFR,0101,1,1978,1978,2226,4,3,1,139933,1984,210012,1984,349945,7000,0,0,0,0,0,Y,349945,7000,342945,,2,34.220327,-118.623062,2358.49,0.0,0.0,0.0,12/02/2024,2358.49,2358.49,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,4716.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016313,90732-4530,LOS ANGELES,7563037052,2024,20,2193 WARMOUTH ST LOS ANGELES CA 90732,SFR,0100,1,1959,1959,1933,3,3,1,485785,1995,263473,1995,749258,7000,0,0,0,0,0,Y,749258,7000,742258,,14,33.719514,-118.325376,4766.57,0.0,0.0,0.0,11/07/2024,4766.57,0.00,0.0,0.0,02/21/2025,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,9533.14
2016314,90732-4519,LOS ANGELES,7563037053,2024,20,3810 S ANCHOVY AVE LOS ANGELES CA 90732,SFR,0100,1,1965,1965,2206,3,2,1,793428,2020,198356,2020,991784,0,0,0,0,0,0,Y,991784,0,991784,,14,33.719952,-118.325128,6241.31,0.0,0.0,0.0,11/27/2024,6241.31,0.00,0.0,0.0,01/16/2025,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,12482.62
2016315,90732-4519,LOS ANGELES,7563037054,2024,20,3806 S ANCHOVY AVE LOS ANGELES CA 90732,SFR,0100,1,1959,1961,1862,3,2,1,703626,2008,175899,2008,879525,7000,0,0,0,0,0,Y,879525,7000,872525,,14,33.720119,-118.325081,5512.25,0.0,0.0,0.0,12/11/2024,5512.25,5512.25,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,11024.50
2016316,90732-4556,LOS ANGELES,7563037055,2024,20,2197 W PASEO DEL MAR LOS ANGELES CA 90732,SFR,0100,1,1962,1962,2153,4,2,1,47779,1975,92991,1975,140770,7000,0,0,0,0,0,Y,140770,7000,133770,,14,33.720279,-118.325034,1

In [12]:
# selected.to_csv("LA_City_Property_Tax_Data.csv")

In [13]:
zoning = gpd.read_file("data/Zoning.geojson")
zoning = zoning[["ZONE_COMPLT", "geometry"]]
zoning

,ZONE_COMPLT,geometry
0,[LF1-WH1-5] [A1-1L],"POLYGON ((-118.24535 34.07054, -118.2455 34.07..."
1,[VF1-WH1-6] [I1-N],"POLYGON ((-118.2239 34.07033, -118.2239 34.070..."
2,[LN1-MU2-5] [P2-FA] [CPIO],"POLYGON ((-118.24139 34.06798, -118.24155 34.0..."
3,[LN1-SH2-5] [RX1-FA] [CPIO],"POLYGON ((-118.23419 34.06842, -118.23428 34.0..."
4,[LM2-MU1-5] [CX2-FA] [CPIO-O],"POLYGON ((-118.24305 34.06724, -118.24302 34.0..."
...,...,...
58883,[Q]C2-2D-CPIO,"POLYGON ((-118.32636 34.0872, -118.32641 34.08..."
58884,C2-2D-SN-CPIO,"POLYGON ((-118.33215 34.09785, -118.33215 34.0..."
58885,[Q]C2-2D-CPIO,"POLYGON ((-118.30238 34.09071, -118.30238 34.0..."
58886,[Q]C2-1XL-CPIO,"POLYGON ((-118.34137 34.08304, -118.34151 34.0..."


In [14]:
# Step 1: Convert the regular dataframe to a GeoDataFrame by creating Point geometries
# First, make a copy to avoid modifying the original
selected_gdf = selected.copy()

# Create a geometry column with Point objects from latitude and longitude
selected_gdf["geometry"] = selected_gdf.apply(
    lambda row: Point(row["Location Longitude"], row["Location Latitude"]), axis=1
)

# Convert to a GeoDataFrame
selected_gdf = gpd.GeoDataFrame(selected_gdf, geometry="geometry")

# Make sure both GeoDataFrames have the same CRS (Coordinate Reference System)
# If you know the CRS of your data, set it explicitly
# For example, if your coordinates are in WGS84:
selected_gdf.crs = "EPSG:4326"

# Ensure zoning has the same CRS, or reproject if needed
if zoning.crs != selected_gdf.crs:
    zoning = zoning.to_crs(selected_gdf.crs)

# Step 2: Perform spatial join - this will add zoning attributes to each point
joined_data = gpd.sjoin(selected_gdf, zoning, how="left", predicate="within")

# Step 3: If you want a regular dataframe with the original columns plus zoning info
# (You might want to drop the extra geometry column and index_right column)
result = pd.DataFrame(joined_data.drop(columns=["geometry", "index_right"]))

# Alternatively, if you want to add specific columns from zoning to your original dataframe:
# selected['zoning_type'] = joined_data['zoning_type']  # Replace with your actual column names

In [15]:
result["ZONE_PREFIX"] = result["ZONE_COMPLT"].str.split("-").str[0]
result

,Zip Code,City Tax Rate Area,AIN,Roll Year,Tax Rate Area Code,Property Location,Property Use Type,Property Use Code,Number of Buildings,Year Built,Effective Year,Square Footage,Number of Bedrooms,Number of Bathrooms,Number of Units,Land Value,Land Base Year,Improvement Value,Improvement Base Year,"Total Value, Land + Improvement",Home Owners Exemption,Real Estate Exemption,Fixture Value,Fixture Exemption,Personal Property Value,Personal Property Exemption,Property taxable?,Total Value,Total Exemption,Taxable Value,Classification,Region Number,Location Latitude,Location Longitude,F1ST_INSTALLMENT_TAX,F1ST_INSTALLMENT_BALANCE_DUE,F1ST_INSTALLMENT_PENALTY,F1ST_INSTALLMENT_PENALTY_PAID,F1ST_INSTALLMENT_PAID_DATE,F2ND_INSTALLMENT_TAX,F2ND_INSTALLMENT_BALANCE_DUE,F2ND_INSTALLMENT_PENALTY,F2ND_INSTALLMENT_PENALTY_PAID,F2ND_INSTALLMENT_PAID_DATE,COST_DUE,COST_PAID,NONSUFFICIENT_FUNDS_DUE,NONSUFFICIENT_FUNDS_PAID,TOTAL_TAX_DUE,TAX_TYPE_1_AMOUNT,TAXTYPE_2_AMOUNT,TAX_TYPE_3_AMOUNT,TAX_TYPE_4_AMOUNT,TAX_TYPE_8_AMOUNT,Total_Taxes_Paid_Calc,ZONE_COMPLT,ZONE_PREFIX,ZONE_Letter
0,91304-3327,LOS ANGELES,2004001003,2024,16,8321 FAUST AVE LOS ANGELES CA 91304,SFR,0101,1,1973,1973,2090,4,3,1,711689,2006,301176,2006,1012865,0,0,0,0,0,0,Y,1012865,0,1012865,,2,34.220225,-118.620681,6326.89,0.0,0.0,0.0,11/05/2024,6326.87,0.00,0.0,0.0,02/03/2025,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,12653.76,RE9-1,RE9,R
1,91304-3327,LOS ANGELES,2004001004,2024,16,8313 FAUST AVE LOS ANGELES CA 91304,SFR,0101,1,1973,1973,2479,5,3,1,370538,2010,255879,2010,626417,0,0,0,0,0,0,Y,626417,0,626417,,2,34.220044,-118.620681,4017.94,0.0,0.0,0.0,12/10/2024,4017.94,4017.94,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,8035.88,RE9-1,RE9,R
2,91304-3327,LOS ANGELES,2004001005,2024,16,8309 FAUST AVE LOS ANGELES CA 91304,SFR,0100,1,1973,1973,2057,4,2,1,526360,2018,198577,2018,724937,0,0,0,0,0,0,Y,724937,0,724937,,2,34.219862,-118.620688,4569.88,0.0,0.0,0.0,11/20/2024,4569.88,4569.88,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,9139.76,RE9-1,RE9,R
3,91304-3332,LOS ANGELES,2004001008,2024,16,8325 MAYNARD AVE LOS ANGELES CA 91304,SFR,0101,1,1978,1978,2423,4,3,1,128421,1980,221965,1980,350386,7000,0,0,0,0,0,Y,350386,7000,343386,,2,34.220339,-118.622718,2354.27,0.0,0.0,0.0,12/11/2024,2354.26,2354.26,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,4708.53,RE11-1,RE11,R
4,91304-3332,LOS ANGELES,2004001009,2024,16,8311 MAYNARD AVE LOS ANGELES CA 91304,SFR,0101,1,1978,1978,2226,4,3,1,139933,1984,210012,1984,349945,7000,0,0,0,0,0,Y,349945,7000,342945,,2,34.220327,-118.623062,2358.49,0.0,0.0,0.0,12/02/2024,2358.49,2358.49,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,4716.98,RE11-1,RE11,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016313,90732-4530,LOS ANGELES,7563037052,2024,20,2193 WARMOUTH ST LOS ANGELES CA 90732,SFR,0100,1,1959,1959,1933,3,3,1,485785,1995,263473,1995,749258,7000,0,0,0,0,0,Y,749258,7000,742258,,14,33.719514,-118.325376,4766.57,0.0,0.0,0.0,11/07/2024,4766.57,0.00,0.0,0.0,02/21/2025,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,9533.14,R1-1XL,R1,R
2016314,90732-4519,LOS ANGELES,7563037053,2024,20,3810 S ANCHOVY AVE LOS ANGELES CA 90732,SFR,0100,1,1965,1965,2206,3,2,1,793428,2020,198356,2020,991784,0,0,0,0,0,0,Y,991784,0,991784,,14,33.719952,-118.325128,6241.31,0.0,0.0,0.0,11/27/2024,6241.31,0.00,0.0,0.0,01/16/2025,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,12482.62,R1-1XL,R1,R
2016315,90732-4519,LOS ANGELES,7563037054,2024,20,3806 S ANCHOVY AVE LOS ANGELES CA 90732,SFR,0100,1,1959,1961,1862,3,2,1,703626,2008,175899,2008,879525,7000,0,0,0,0,0,Y,879525,7000,872525,,14,33.720119,-118.325081,5512.25,0.0,0.0,0.0,12/11/2024,5512.25,5512.25,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,11024.50,R1-1XL,R1,R
2016316,90732-4556,LOS ANGELES,7563037055,2024,20,2197 W PASEO DEL M

In [17]:
result["ZONE_PREFIX"].value_counts()

ZONE_PREFIX
R1           251579
R3            61505
RS            55093
R2            51851
RD1.5         36204
              ...  
C1(PV)            1
[T]M2             1
M2(PV)            1
[Q]CM(GM)         1
[T]RE9            1
Name: count, Length: 325, dtype: int64

In [18]:
parcels = gpd.read_file("data/Parcels.gdb", columns=["AIN", "geometry"])

In [20]:
parcels["AIN"] = pd.to_numeric(parcels["AIN"], errors="coerce").astype(
    "Int64"
)  # Keeps NaNs


In [21]:
merged_results = pd.merge(
    result,
    parcels,
    on="AIN",  # Join on the AIN column that exists in both dataframes
    how="left",  # Keep all rows from 'result' even if no matching AIN in parcels
)

# If you want to convert the merged result back to a GeoDataFrame
# (in case you need to do more spatial operations later)
# Note: This will use the geometry from the parcels dataframe
merged_gdf = gpd.GeoDataFrame(merged_results, geometry="geometry")

In [22]:
projected_gdf = merged_gdf.to_crs("EPSG:2229")

# Calculate area in square feet first (State Plane uses US Survey Feet)
projected_gdf["area_sq_feet"] = projected_gdf.geometry.area

# Convert to acres (1 acre = 43,560 square feet)
projected_gdf["acreage"] = projected_gdf["area_sq_feet"] / 43560

# Drop the intermediate area calculation if you don't need it
projected_gdf = projected_gdf.drop(columns=["area_sq_feet"])

# If you want to round the acreage to a specific number of decimal places
projected_gdf["acreage"] = projected_gdf["acreage"].round(3)

# Convert back to original CRS if needed for further spatial operations
final_gdf = projected_gdf.to_crs(merged_gdf.crs)

In [24]:
final_gdf[final_gdf["Property Location"].str.startswith("1017 HYPERION AVE", na=False)]


,Zip Code,City Tax Rate Area,AIN,Roll Year,Tax Rate Area Code,Property Location,Property Use Type,Property Use Code,Number of Buildings,Year Built,Effective Year,Square Footage,Number of Bedrooms,Number of Bathrooms,Number of Units,Land Value,Land Base Year,Improvement Value,Improvement Base Year,"Total Value, Land + Improvement",Home Owners Exemption,Real Estate Exemption,Fixture Value,Fixture Exemption,Personal Property Value,Personal Property Exemption,Property taxable?,Total Value,Total Exemption,Taxable Value,Classification,Region Number,Location Latitude,Location Longitude,F1ST_INSTALLMENT_TAX,F1ST_INSTALLMENT_BALANCE_DUE,F1ST_INSTALLMENT_PENALTY,F1ST_INSTALLMENT_PENALTY_PAID,F1ST_INSTALLMENT_PAID_DATE,F2ND_INSTALLMENT_TAX,F2ND_INSTALLMENT_BALANCE_DUE,F2ND_INSTALLMENT_PENALTY,F2ND_INSTALLMENT_PENALTY_PAID,F2ND_INSTALLMENT_PAID_DATE,COST_DUE,COST_PAID,NONSUFFICIENT_FUNDS_DUE,NONSUFFICIENT_FUNDS_PAID,TOTAL_TAX_DUE,TAX_TYPE_1_AMOUNT,TAXTYPE_2_AMOUNT,TAX_TYPE_3_AMOUNT,TAX_TYPE_4_AMOUNT,TAX_TYPE_8_AMOUNT,Total_Taxes_Paid_Calc,ZONE_COMPLT,ZONE_PREFIX,ZONE_Letter,geometry,acreage
618936,90029-3109,LOS ANGELES,5427007013,2024,13,1017 HYPERION AVE LOS ANGELES CA 90029,R-I,0200,2,1946,1952,2184,6,2,2,1096500,2023,418200,2023,1514700,0,0,0,0,0,0,Y,1514700,0,1514700,,4,34.090412,-118.280831,9291.82,0.0,0.0,0.0,11/13/2024,9291.8,9291.8,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,18583.62,R3-1VL,R3,R,"MULTIPOLYGON (((6476718.72 1855390.06, 6476695...",0.168


In [25]:
final_gdf["Property_Tax_Value"] = final_gdf["Taxable Value"] * 0.01

In [34]:
final_gdf = final_gdf.dropna(subset=["acreage"])

In [35]:
final_gdf[final_gdf["acreage"].isna()]


,Zip Code,City Tax Rate Area,AIN,Roll Year,Tax Rate Area Code,Property Location,Property Use Type,Property Use Code,Number of Buildings,Year Built,Effective Year,Square Footage,Number of Bedrooms,Number of Bathrooms,Number of Units,Land Value,Land Base Year,Improvement Value,Improvement Base Year,"Total Value, Land + Improvement",Home Owners Exemption,Real Estate Exemption,Fixture Value,Fixture Exemption,Personal Property Value,Personal Property Exemption,Property taxable?,Total Value,Total Exemption,Taxable Value,Classification,Region Number,Location Latitude,Location Longitude,F1ST_INSTALLMENT_TAX,F1ST_INSTALLMENT_BALANCE_DUE,F1ST_INSTALLMENT_PENALTY,F1ST_INSTALLMENT_PENALTY_PAID,F1ST_INSTALLMENT_PAID_DATE,F2ND_INSTALLMENT_TAX,F2ND_INSTALLMENT_BALANCE_DUE,F2ND_INSTALLMENT_PENALTY,F2ND_INSTALLMENT_PENALTY_PAID,F2ND_INSTALLMENT_PAID_DATE,COST_DUE,COST_PAID,NONSUFFICIENT_FUNDS_DUE,NONSUFFICIENT_FUNDS_PAID,TOTAL_TAX_DUE,TAX_TYPE_1_AMOUNT,TAXTYPE_2_AMOUNT,TAX_TYPE_3_AMOUNT,TAX_TYPE_4_AMOUNT,TAX_TYPE_8_AMOUNT,Total_Taxes_Paid_Calc,ZONE_COMPLT,ZONE_PREFIX,ZONE_Letter,geometry,acreage,Property_Tax_Value


In [31]:
final_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 805748 entries, 0 to 805747
Data columns (total 61 columns):
 #   Column                           Non-Null Count   Dtype   
---  ------                           --------------   -----   
 0   Zip Code                         805748 non-null  object  
 1   City Tax Rate Area               805748 non-null  object  
 2   AIN                              805748 non-null  int64   
 3   Roll Year                        805748 non-null  int64   
 4   Tax Rate Area Code               805748 non-null  int64   
 5   Property Location                766878 non-null  object  
 6   Property Use Type                805748 non-null  object  
 7   Property Use Code                805748 non-null  object  
 8   Number of Buildings              805748 non-null  int64   
 9   Year Built                       805748 non-null  int64   
 10  Effective Year                   805748 non-null  int64   
 11  Square Footage                   805748 non-

In [37]:
final_gdf["Property_Tax_Value"].sum()


np.float64(8078718255.569998)

In [44]:
# Function to extract first letter of zoning code
def extract_first_letter(zone_code):
    if pd.isna(zone_code):
        return None

    # Remove any brackets or parentheses and their contents
    # This regex looks for patterns like [...], (...), etc.
    cleaned_code = re.sub(r"[\[\(].*?[\]\)]", "", zone_code)

    # Remove any remaining special characters
    cleaned_code = re.sub(r"[^a-zA-Z0-9]", "", cleaned_code)

    # Take the first letter if there is one
    if cleaned_code and len(cleaned_code) > 0:
        return cleaned_code[0]
    else:
        return None


# Apply the function to create a new column
final_gdf["zone_type"] = final_gdf["ZONE_PREFIX"].apply(extract_first_letter)


/Users/joeyshoyer/Desktop/SFA/parcels/.venv/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [46]:
final_gdf["zone_type"].value_counts()

zone_type
R    697050
C     53791
M     17263
P      7582
H      6522
D      4399
L      4069
Q      3891
O      3431
A      3273
U       726
N       353
V        51
T        48
G        30
F         8
S         2
Name: count, dtype: int64

In [ ]:
# final_gdf.to_csv("LA_City_Property_Parcel_Acreage_Zoning_Tax_Data.csv")

In [42]:
# Group by ZONE_PREFIX and calculate aggregates
zone_summary = {}

# Get unique zone prefixes
zone_prefixes = final_gdf["ZONE_PREFIX"].unique()
print(zone_prefixes)
for zone in zone_prefixes:
    # Filter for just this zone prefix
    zone_data = final_gdf[final_gdf["ZONE_PREFIX"] == zone]

    # Skip if zone is None/NaN
    if pd.isna(zone):
        continue

    try:
        # Calculate metrics with error handling
        total_acreage = zone_data["acreage"].sum()
        total_property_tax = zone_data["Property_Tax_Value"].sum()
    except KeyError as e:
        print(f"Missing key {e} in DataFrame. Skipping zone: {zone}")
        continue  # Skip this zone if keys are missing

    # Avoid division by zero
    if total_acreage > 0:
        tax_per_acre = total_property_tax / total_acreage
    else:
        tax_per_acre = 0

    # Store in dictionary
    zone_summary[zone] = {
        "total_acreage": round(total_acreage, 2),
        "total_property_tax": round(total_property_tax, 2),
        "tax_per_acre": round(tax_per_acre, 2),
    }

# Now zone_summary is a dictionary with all the metrics by zone prefix
print(zone_summary)

['RE9' 'RE11' 'OS' '[Q]OS' '(Q)R1' '(Q)RE11' '(Q)RD3' '[T][Q]M1' 'A1' 'RS'
 'C2' '[Q]M1' 'R3' 'C4' 'P' '[Q]PF' '(T)(Q)C1.5' 'CR' nan '(T)(Q)R4'
 '(Q)R3' '(T)RS' 'R1' 'RA' '[Q]C1' '[Q]C1.5' '(Q)RD5' 'RE20' 'RE40' 'RE15'
 '(T)RE11' 'A2' 'QCR' '(T)(Q)CR' 'QC1' '(Q)RD2' 'QRD3' '[Q]CR' '[Q]C2'
 'C1' '(T)(Q)C2' 'R1P' 'C1.5' 'R2' '(Q)C1.5' '(WC)TOPANGA' '(T)(Q)RD2'
 'RD2' 'RD1.5' '(Q)RD1.5' '(Q)RD6' '(T)(Q)R1' '(Q)CR' '(T)(Q)RD3'
 '(T)(Q)RD5' '(Q)C1' '(Q)C4' '[Q]C4' 'QC2' '(Q)P' '(Q)C2' '(T)(Q)C1' 'PF'
 '(T)R1' '[Q]R3' '(T)R3' 'QRD5' 'R4' '(T)RE9' '(T)(Q)RD4' '(Q)RE9'
 '(T)(Q)R3' 'M2' 'MR1' '(Q)MR1' '[Q]CM' '[T][Q]C2' '(T)(Q)RZ3' '[Q]RD2'
 '[Q]P' '(Q)R3P' '(T)(Q)RD1.5' '(Q)M1' 'RD5' 'M1' 'RD3' '(T)(Q)M1' 'QR3'
 '(Q)RS' '[T]R3' 'QRD1.5' 'R3P' '(Q)R4' '(Q)RZ2.5' '(Q)R2' '[Q]R2' '[Q]R1'
 '[Q]RAS3' '(Q)RD4' '(T)[Q]C2' '[T]R1' '(T)[Q]R4' '(T)(Q)RAS4' '[Q]RD5'
 '(Q)RAS3' '(T)(Q)RAS3' '(T)[Q]RAS4' 'RMP' '[T]RD2' '[T][Q]RD1.5' 'QRD2'
 '(T)(Q)C4' '[Q]RD3' '(WC)RIVER' '(Q)RAS4' '[T][Q]MR1' '[Q]MR1'
 '(

In [47]:
# Group by ZONE_PREFIX and calculate aggregates
zone_summary_types = {}

# Get unique zone prefixes
zone_prefixes = final_gdf["zone_type"].unique()
print(zone_prefixes)
for zone in zone_prefixes:
    # Filter for just this zone prefix
    zone_data = final_gdf[final_gdf["zone_type"] == zone]

    # Skip if zone is None/NaN
    if pd.isna(zone):
        continue

    try:
        # Calculate metrics with error handling
        total_acreage = zone_data["acreage"].sum()
        total_property_tax = zone_data["Property_Tax_Value"].sum()
    except KeyError as e:
        print(f"Missing key {e} in DataFrame. Skipping zone: {zone}")
        continue  # Skip this zone if keys are missing

    # Avoid division by zero
    if total_acreage > 0:
        tax_per_acre = total_property_tax / total_acreage
    else:
        tax_per_acre = 0

    # Store in dictionary
    zone_summary_types[zone] = {
        "total_acreage": round(total_acreage, 2),
        "total_property_tax": round(total_property_tax, 2),
        "tax_per_acre": round(tax_per_acre, 2),
    }

# Now zone_summary is a dictionary with all the metrics by zone prefix
print(zone_summary_types)

['R' 'O' 'M' 'A' 'C' 'P' None 'Q' 'T' 'U' 'D' 'N' 'L' 'H' 'F' 'V' 'G' 'S']
{'R': {'total_acreage': np.float64(405531.82), 'total_property_tax': np.float64(5765060693.26), 'tax_per_acre': np.float64(14216.05)}, 'O': {'total_acreage': np.float64(39112.82), 'total_property_tax': np.float64(979096.22), 'tax_per_acre': np.float64(25.03)}, 'M': {'total_acreage': np.float64(21963.02), 'total_property_tax': np.float64(440195874.7), 'tax_per_acre': np.float64(20042.59)}, 'A': {'total_acreage': np.float64(22128.34), 'total_property_tax': np.float64(33530500.13), 'tax_per_acre': np.float64(1515.27)}, 'C': {'total_acreage': np.float64(32026.78), 'total_property_tax': np.float64(1230219587.52), 'tax_per_acre': np.float64(38412.22)}, 'P': {'total_acreage': np.float64(19584.89), 'total_property_tax': np.float64(47433507.91), 'tax_per_acre': np.float64(2421.94)}, 'Q': {'total_acreage': np.float64(15893.74), 'total_property_tax': np.float64(26300495.05), 'tax_per_acre': np.float64(1654.77)}, 'T': {'tot

In [41]:
zone_summary["R1"]

{'total_acreage': np.float64(41356.13),
 'total_property_tax': np.float64(1606606539.78),
 'tax_per_acre': np.float64(38848.09)}

In [48]:
zone_summary_types["R"]

{'total_acreage': np.float64(405531.82),
 'total_property_tax': np.float64(5765060693.26),
 'tax_per_acre': np.float64(14216.05)}